In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train_data = pd.read_csv('/content/drive/MyDrive/datasets/subtask_1/es/train.tsv',sep='\t')
train_data = train_data.reset_index(drop=True)
print(train_data.head())

      id                                               text      label
0   5464  Entrada en vigor. La presente Directiva entrar...      human
1  30129  Preguntas: 1. ¿Cuáles son los principales argu...  generated
2  19553  ¿Desea algo? Póngame una caja de madera. ¿Qué ...  generated
3  13005  @victor28088 1665 Tweets no originales, que as...      human
4  16919  De pequeño Dios me dio a elegir entre tener un...      human


In [5]:
from sklearn.model_selection import train_test_split
train_data_texts = train_data['text'].to_list()
train_data_labels = train_data['label'].to_list()
train_texts, test_texts, train_labels, test_labels = train_test_split(train_data_texts, train_data_labels, test_size=0.1)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.1)
print('train data size: ', len(train_texts))
print('validation data size: ', len(val_texts))
print('test data size: ', len(test_texts))


train data size:  25969
validation data size:  2886
test data size:  3207


In [6]:
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

In [16]:
import nltk
# feature augmentation using pos tags
nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
tagger = nltk.pos_tag

train_texts_pos = []
for text in train_texts:
    # Tokenize text and get POS tags
    tokens = tokenizer.tokenize(text)
    train_pos_tags = [tag for token, tag in nltk.pos_tag(tokens)]
    train_texts_pos.append(train_pos_tags)

val_texts_pos = []
for text in val_texts:
    # Tokenize text and get POS tags
    tokens = tokenizer.tokenize(text)
    val_pos_tags = [tag for token, tag in nltk.pos_tag(tokens)]
    val_texts_pos.append(val_pos_tags)

test_texts_pos = []
for text in test_texts:
    # Tokenize text and get POS tags
    tokens = tokenizer.tokenize(text)
    test_pos_tags = [tag for token, tag in nltk.pos_tag(tokens)]
    test_texts_pos.append(test_pos_tags)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [14]:
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased").to(device)
print("Model Configurations")
print()
print(model.config)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Model Configurations

BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}



In [18]:
# Convert to a list of strings
train_texts_pos = [' '.join(pos_tags) for pos_tags in train_texts_pos]
val_texts_pos = [' '.join(pos_tags) for pos_tags in val_texts_pos]
test_texts_pos = [' '.join(pos_tags) for pos_tags in test_texts_pos]

train_encodings = tokenizer(train_texts, train_texts_pos, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, val_texts_pos, truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_texts, test_texts_pos, truncation=True, padding=True, max_length=256)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels[idx] == 'human':
            item['labels'] = torch.tensor(0)
        else:
            item['labels'] = torch.tensor(1)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

print("Sample Data Point")
print()
print(train_dataset[0])

Sample Data Point

{'input_ids': tensor([    4,  1094,  1009, 17487,  1409,  4376,  1181,  1162,  4376,  1058,
         1108,  7730,  1785,  1008, 15642,  1030, 10661,  1042,  1030, 19748,
         1017,  1042,  1447,  2353,  1013, 22260,  1072, 15401,  1009,  3608,
         6824,  1051,  9602,  1017,  7950,  1017, 12329,  1036, 15010,  1074,
        28558,  1110, 29157,  3695,  1042, 12588,  1066,  1072, 15401,  1009,
         1115,  1009, 27737, 30932,  5648,  1008, 18170,  1181,  1198, 18170,
        10236,  1058,  1108,  7730,  1785,  1008, 21345,  1040,  2116, 10151,
         1042, 19966,  1030, 19748,  1009, 16339,  1013, 24588, 30936,  1042,
        24923,  1018, 14527,  1017, 14637,  1597,     5,  7877,  1009,  1083,
        30969, 30966,  1083, 30969,  1083, 30969,  1181,  1083, 30969, 30966,
         1171, 30981,   983,  1195,   987,   993,   992,  1083, 30969,  1083,
        30969,  4332,  1195,   987,  1195,   987,  1195,   987,  1195,   987,
         1195,   987,  1195,   

In [9]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [19]:
training_args = TrainingArguments(
    output_dir='./results',       
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    evaluation_strategy='epoch',
    save_strategy='epoch',
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.291300,0.240740,0.896743,0.896499,0.905276,0.899058
2,0.162000,0.349297,0.908870,0.908705,0.916209,0.911017
3,0.016300,0.347458,0.933125,0.933116,0.935265,0.934364


TrainOutput(global_step=2436, training_loss=0.15886281055554272, metrics={'train_runtime': 3573.8109, 'train_samples_per_second': 21.799, 'train_steps_per_second': 0.682, 'total_flos': 1.024909649496576e+16, 'train_loss': 0.15886281055554272, 'epoch': 3.0})

In [20]:
trainer.evaluate(test_dataset)
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.97      0.89      0.93      1568
           1       0.90      0.97      0.94      1639

    accuracy                           0.93      3207
   macro avg       0.94      0.93      0.93      3207
weighted avg       0.94      0.93      0.93      3207

